# Automatic door control and motion logger

---

## Aim

* This application notebook uses Grove servo and Grove PIR motion sensor to create an Automatic door control system. When the PIR sensor detects a movement the servo will open the door. In the second part of the notebook we analyse the data collected from motion sensor and plot a 24/7 time wheel which depicts a heatmap showing the busy hours of people entering through the automatic door.

## References
* [Jupyterplot](https://lvwerra.github.io/jupyterplot/)
* [Grove PIR](https://www.seeedstudio.com/Grove-PIR-Motion-Sensor.html)   
* [Grove Servo](https://wiki.seeedstudio.com/Grove-Servo.html)
* [Grove I2C OLED](https://wiki.seeedstudio.com/Grove-OLED_Display_0.96inch/)
* [Grove Base Shield V2.0](https://www.seeedstudio.com/Base-Shield-V2.html)   
* [time-wheel-in-python3-pandas](https://stackoverflow.com/questions/40352607/time-wheel-in-python3-pandas/49010015)

## Last revised
* 20 April 2021
    + Initial version
---

## Import libraries and load _base_ Overlay

In [ ]:
import datetime
import random
from collections import OrderedDict
from time import sleep

import pandas as pd
import pytz
from pynq.overlays.base import BaseOverlay
from pynq_peripherals import ArduinoSEEEDGroveAdapter
from pytz import timezone

base = BaseOverlay('base.bit')

<div class="alert alert-box alert-info">
   <h4 class="alert-heading">Library Dependencies </h4>
Before proceeding the following libraries must be installed
    <ul>
        <li> Jupyterplot </li>
    </ul>
</div>

In [ ]:
try:
    from jupyterplot import ProgressPlot
except ImportError:
    print("This script requires the jupyterplot module \nInstall with: "
          "sudo pip3 install jupyterplot")

## Constructing application with Grove Base Shield V2.0 (Arduino)

<div class="alert alert-box alert-warning">    
   <h4 class="alert-heading">Make Physical Connections </h4>
    <ul>
        <li>Insert the Grove Base Shield into the Arduino connector on the board.</li>
        <li>Connect the grove_servo module to D5 connector of the Grove Base Shield.</li>
        <li>Connect the grove_pir module to D6 connector of the Grove Base Shield.</li>
        <li> Connect the grove_oled module to I2C connector of the Grove Base Shield.</li></ul>
</div>

![](images/door_control.png)

### Adapter configuration

In [ ]:
adapter = ArduinoSEEEDGroveAdapter(base.ARDUINO, D6='grove_pir', D5='grove_servo', I2C='grove_oled')

### Define device objects

In [ ]:
motion_sensor = adapter.D6
servo = adapter.D5
oled = adapter.I2C

### Construct the automatic door controller

In [ ]:
def open_door():
    for x in range(10, 170, 10):
        servo.set_angular_position(x)
        sleep(0.01)
    for x in range(160, 0, -10):
        servo.set_angular_position(x)
        sleep(0.1)


<div class="alert alert-box alert-info">
   <h4 class="alert-heading">Notes before running the next cell </h4>
    <ul>
        <li> Toggle Switch SW0 on the PYNQ-Z2 board is used to enable or disable the motion sensor.</li>
        <li> Detection data is being collected while running this cell. To exit the cell just Interrupt the kernel with the stop sign on the tool bar </li>
    </ul>
</div>

In [ ]:
# define counter
counter = 0

oled.set_default_config()
oled.clear_display()
detect_time = list()
capture = list()

pp = ProgressPlot(plot_names=['Detection frequency'],
                  line_names=['Door opened'], line_colors=None,
                  x_lim=[None, None], y_lim=[0, 1],
                  x_label='detection attempts', x_iterator=True, height=None,
                  width=600);

try:
    for i in range(1000):
        oled.set_position(0, 0)
        oled.put_string('The person')
        oled.set_position(2, 0)
        oled.put_string('number is:')
        oled.set_position(4, 0)
        if base.switches[0].read():
            if motion_sensor.motion_detected():
                detect_time.append(
                    datetime.datetime.now(tz=pytz.timezone('US/Pacific')))
                capture.append(1)
                counter += 1
                oled.put_string(str(counter))
                d = {'detect_time': detect_time, 'capture': capture}
                df = pd.DataFrame(data=d)
                pp.update(1)
                pp.update(0)
                open_door()
                sleep(1)
            else:
                oled.put_string(str(counter))
                detect_time.append(
                    datetime.datetime.now(tz=pytz.timezone('US/Pacific')))
                capture.append(0)
                pp.update(0)
                pp.update(0)
        sleep(0.1)
except KeyboardInterrupt:
    print('Stopping the loop')
    
pp.finalize()


### Sample the collected data

In [ ]:
df.head()

### Logging movements and weekly table

In [ ]:
# generate the table with timestamps
times = df.detect_time.where(df.capture == 1).dropna()
times.name = None
data = pd.crosstab(times.dt.strftime("%A"), times.dt.hour.apply(
    lambda x: '{:02d}:00'.format(x))).fillna(0)

test = [str(i) + ':00' if i > 9 else '0' + str(i) + ':00' for i in range(24)]

# initialize data of lists.
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
dlist = OrderedDict(dict(zip(days, [([0] * 24) for _ in range(7)])))

# Create DataFrame
df_master = pd.DataFrame(dlist, index=test)

df_weekly = df_master.T
for i in test:
    for j in dlist.keys():
        exists = i in data
        if exists:
            exists1 = j in data[i]
            if exists1:
                df_weekly[i].loc[j] = data[i].loc[j]

print(df_weekly.T)


<div class="alert alert-box alert-info">
   <p class="alert-heading">Note: Below we show simulated table for one full week of detections for demonstration purpose </p>
</div>

In [ ]:
test_sim = [str(i) + ':00' if i > 9 else '0' + str(i) + ':00' for i in range(24)]

# initialize data of lists.
dlist_sim = OrderedDict(dict(zip(days, [random.sample(range(0, 50), 24) for _ in range(7)])))

# Create DataFrame
df_master = pd.DataFrame(dlist_sim, index=test_sim)

df_weekly_sim = df_master.T

print(df_weekly_sim.T)

Copyright (C) 2021 Xilinx, Inc

SPDX-License-Identifier: BSD-3-Clause

----

----